This is an updated version of this file, with the suggestions done by cyang-kth already implemented. 
https://github.com/cyang-kth/fmm/issues/70

# imports

In [1]:
from pylab import *
import pandas as pd
import geopandas as gpd
import networkx as nx

%config Completer.use_jedi = False 
# %matplotlib notebook
%matplotlib inline

import myMatcher as MM

import importlib
importlib.reload(MM)

<module 'myMatcher' from '/home/christianweber/python_GIT/test_fmm/test_fmm/myMatcher.py'>

# collect some functions

In [2]:
def mm_load(shp_):
    importlib.reload(MM)
    mm = MM.myMatcher(p_unx='./', p_nw='./', crs=4326)
    mm.load_network(shp_ = shp_)
    mm.load_data_shp("./data/syntetic_data_ANV_WGS84.shp", id_='track_id')
#     mm.load_data_shp("./data/syntetic_data_ANV_UTM.shp")
    mm.generate_lineString()
    return mm

def config_match_plot(mm, k=8, d=10, radius=20, factor=15, gps_error=50, with_params=False):
    #config
    if d>0:
        mm.reduce_trajectory(d=d)
        mm.wkt = mm.gdf_lineString.iloc[0].ls_ip.wkt
    else:
        mm.wkt = mm.gdf_lineString.iloc[0].geometry.wkt
    mm.config_matcher(radius=radius, k=k, factor=factor, gps_error=gps_error, )
    #match
    mm.match_wkt()
    #plot
#     mm.plot(zoom_=True, with_params=with_params)    
    return mm

# directed vs not directed network

In [3]:
# shp_directed = 'data/bike_centerline_v1_extract_ANVedges_source_target.shp'
# shp_undirected = 'data/bike_centerline_v1_extract_ANVedges_source_target_rev.shp'
# shp_undirected = 'data/bike_centerline_v1_extract_ANVedges_source_target_rev_renamed.shp'
# shp_undirected = 'data/bike_stat_centerline_2_extract_ANV_miniedges_source_target_rev.shp'
shp_undirected = 'data/bike_centerline_v1_extract_ANVedges_source_target_rev_renamed_mini_WGS84.shp'

# shp_ = shp_directed
shp_ = shp_undirected

In an earlier version of this work, I only swapped source-target in the network shapefile, in order to generate the "undirected" network. This, however, does not affect the direction of the lineString itself, so STMATCH was trying to match an impossible network (and failed, naturally). <br>
For reference, both networks are stored here, and we can observe the behavior by loading the directed (i.e. only one direction per lineString, but source-targed swapped) and the undirected network (source-target not swapped, but lineString direction reversed in the second half of the dataframe). 

# Network topology

The network is bidirectional, as an example we look at one edge:

In [4]:
# mm.gdf_nw.rename(columns={'id': 'id_singleSet', 'uid': 'id'}, inplace=True)

In [5]:
# mm.gdf_nw.to_file(mm.p_nw + 'data/bike_centerline_v1_extract_ANVedges_source_target_rev_renamed.shp')

In [6]:
mm = mm_load(shp_)
mm.gdf_nw.loc[mm.gdf_nw.id == 9008]

Loaded network: 28 nodes, 60 edges


,uid,Shape_Leng,id,slope,source,target,ShpName,cost,geometry
3,3,15.397967,9008,0.064944,2,4,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86592 59.95544, 10.86589 59.955..."
33,33,15.397967,9008,0.064944,2,4,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86566 59.95545, 10.86568 59.955..."


Note: for `shp_directed`, we see that source-target is swapped, but the geometry is the same lineString. For `shp_undirected`, source-target is identical, but each lineString is reversed. 

# test parameters

So far, I don't get a perfect fit with the STMATCH algorithm. <br>As stated above, the network should be ok. (side question: how is the visualization (e.g. here: https://github.com/cyang-kth/fmm/issues/30) done? <br>
__correction:__<br>
The network was __not__ ok. Was missing true bidirectional links, as stated above. And the visualization in QGis is simply done by replacing the lines with arrows - QGis respects the "direction" of the line as the list of coordinates.

## test with WGS84

In [7]:
# #Parameters UTM
# # d=0
# k = 8
# radius=20
# gps_e = 50
# vmax = 30
# factor=15

#Parameters WGS84
# d=0
k = 4
radius=0.4
gps_e = 0.5
vmax = 30
factor=15

In [8]:
mm.gdf_nw.head()

,uid,Shape_Leng,id,slope,source,target,ShpName,cost,geometry
0,0,90.068473,8296,0.022205,0,1,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86825 59.95613, 10.86821 59.956..."
1,1,67.579317,8649,0.029595,1,2,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86703 59.95566, 10.86698 59.955..."
2,2,13.847463,8650,0.000000,1,3,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86703 59.95566, 10.86703 59.955..."
3,3,15.397967,9008,0.064944,2,4,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86592 59.95544, 10.86589 59.955..."
4,4,87.319449,9009,0.103070,3,5,bike_stat_centerline_2_extract_ANV_mini,1,"LINESTRING (10.86687 59.95575, 10.86686 59.955..."


In [9]:
mm.gdf_lineString

,id,geometry,wkt,ls_ip
0,123,"LINESTRING (10.86311 59.95676, 10.86335 59.956...",LINESTRING (10.8631110194893470 59.95676254309...,None


In [10]:
mm = mm_load(shp_)
for i in [0]:
    mm = config_match_plot(mm, d=i, radius=radius, factor=factor, with_params=True) 

Loaded network: 28 nodes, 60 edges


In [11]:
print ("Matched path: ", list(mm.result.cpath))
print ("Matched edge for each point: ", list(mm.result.opath))
print ("Matched edge index ",list(mm.result.indices))
print ("Matched geometry: ",mm.result.mgeom.export_wkt())
print ("Matched point ", mm.result.pgeom.export_wkt())

Matched path:  []
Matched edge for each point:  [12149, 12149, 10902, 10902, 10902, 10902, 10902, 10902, 10122, 10122, 10122, 10122, 9765, 9765, 9765, 9009, 9009, 9380, 9764, 9009, 9009, 9009, 9009, 9009, 8650, 9009]
Matched edge index  []
Matched geometry:  LINESTRING()
Matched point  LINESTRING(10.863 59.9566,10.8633 59.9565,10.8637 59.9564,10.864 59.9564,10.8643 59.9565,10.8643 59.9565,10.8644 59.9565,10.8645 59.9565,10.8647 59.9565,10.8647 59.9565,10.8648 59.9565,10.8649 59.9565,10.8652 59.9565,10.8652 59.9565,10.8656 59.9564,10.8658 59.9562,10.8658 59.9562,10.8657 59.9563,10.8657 59.9564,10.8661 59.9561,10.8661 59.9561,10.8661 59.9561,10.8663 59.956,10.8665 59.9559,10.8669 59.9558,10.8667 59.9559)


## interpolation

In [ ]:
# #Parameters UTM
# # d=0
# k = 8
# radius=20
# gps_e = 50
# vmax = 30
# factor=15

#Parameters WGS84
# d=0
k = 4
radius=0.4
gps_e = 0.5
vmax = 30
factor=15

Vary the interpolation distance:

In [ ]:
mm = mm_load(shp_)
for i in [1, 10, 50, 100]:
    mm = config_match_plot(mm, d=i, radius=radius, factor=factor, with_params=True) 

- Variation of the interpolation distance leads to missing links in the resulting track - __why?__
- All matches include reverse movements.
- The original data is matched best, but with reverse movements. 

## radius

In [ ]:
d=0
k = 8
# radius=20
gps_e = 50
vmax = 30
factor=15

Vary the radius distance:

In [ ]:
mm = mm_load(shp_)
for i in [10, 30, 60, 100, 200]:
    mm = config_match_plot(mm, d=d, radius=i, factor=factor, with_params=True) 

- If the radius is too small, no match is found (r=10)
- A radius of 30 matches these data well, but creates one reverse movement (r=30)
- All larger radii result in missing links - __why?__ (r>30)

## gps error

In [ ]:
d=0
k = 8
radius=20
# gps_e = 50
vmax = 30
factor=15

Vary the interpolation distance:

In [ ]:
mm = mm_load(shp_)
for i in [5, 10, 50, 100]:
    mm = config_match_plot(mm, d=d, radius=radius, gps_error=i, with_params=True) 

- missing links for small gps errors (gps_e < 50)
- reverse movements in all cases

## factor

In [ ]:
d=0
k = 8
radius=20
gps_e = 50
vmax = 30
# factor=15

Vary the factor:

In [ ]:
mm = mm_load(shp_)
for i in [0.5, 1.5, 10, 50, 100]:
    mm = config_match_plot(mm, d=d, radius=radius, factor=i, with_params=True) 

- The factor seems to have little influence in this example. All factors f > 0.5 result in connected matches
- reverse movement in all cases

## k

In [ ]:
d=0
# k = 8
radius=20
gps_e = 50
vmax = 30
factor=15

Vary the number of candidates:

In [ ]:
mm = mm_load(shp_)
for i in [1, 2, 5, 10, 100]:
    mm = config_match_plot(mm, k=i, d=d, radius=radius, factor=factor, with_params=True) 

- k <= 2 results in non-connected match
- k >= 5: connected matches with reverse movement
- k = 5 leads to the most links with reverse movement. 

# Questions

- What is the best way to tune the parameters? 
- Maybe build a minimizer around them? Some kind of gradient search?
- How can the reverse movements be avoided? -> use undirected network!
- Is there any paper available for the STMATCH algorithm? 
- In the FMM-paper, a penalty is mentioned for avoiding reverse movement. Is this still implemented? 
- Will FMM with a precalculated UBODT file give better results? 

are answered by cyang-kth here: https://github.com/cyang-kth/fmm/issues/70